In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Make plots look nicer
sns.set_theme(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["figure.dpi"] = 120

In [ ]:
df = pd.read_csv("Access_to_Everyday_Life_Dataset.csv")
df.head(5)

In [ ]:
df = df.rename(columns={
    "geometry/coordinates/0": "lon",
    "geometry/coordinates/1": "lat",
    "properties/attribute_id": "attribute_id",
    "properties/label_type": "label_type",
    "properties/neighborhood": "neighborhood",
    "properties/severity": "severity",
    "properties/is_temporary": "is_temporary",
    "geometry/type": "geometry_type",
    "type": "feature_type"
})

df.head()


In [ ]:
df["severity"] = pd.to_numeric(df["severity"], errors="coerce")

# Sometimes these come as TRUE/FALSE strings
if df["is_temporary"].dtype == "object":
    df["is_temporary"] = df["is_temporary"].astype(str).str.upper().map({"TRUE": True, "FALSE": False})

df[["severity", "is_temporary"]].dtypes


In [ ]:
(df.isna().mean().sort_values(ascending=False) * 100).round(2)


In [ ]:
#Most Common Barrier Types
top = df["label_type"].value_counts().head(15)

plt.figure()
sns.barplot(x=top.values, y=top.index)
plt.title("Top Barrier Types (Count)")
plt.xlabel("Count")
plt.ylabel("")
plt.show()

In [ ]:
#Severity distribution 
plt.figure()
sns.histplot(df["severity"].dropna(), bins=10, kde=True)
plt.title("Distribution of Severity")
plt.xlabel("Severity")
plt.ylabel("Count")
plt.show()


In [ ]:
#Severity by barrier type
top_types = df["label_type"].value_counts().head(10).index
tmp = df[df["label_type"].isin(top_types)].copy()

plt.figure(figsize=(14, 7))
sns.boxplot(data=tmp, x="label_type", y="severity")
plt.title("Severity by Barrier Type (Top 10)")
plt.xlabel("")
plt.ylabel("Severity")
plt.xticks(rotation=35, ha="right")
plt.show()


In [ ]:
#Temporary vs permanent comparison
plt.figure()
sns.countplot(data=df, x="is_temporary")
plt.title("Temporary vs Permanent Barriers")
plt.xlabel("Is Temporary?")
plt.ylabel("Count")
plt.show()

plt.figure()
sns.boxplot(data=df, x="is_temporary", y="severity")
plt.title("Severity: Temporary vs Permanent")
plt.xlabel("Is Temporary?")
plt.ylabel("Severity")
plt.show()


In [ ]:
#Top neighborhoods by number of barriers
top_n = df["neighborhood"].value_counts().head(15)

plt.figure(figsize=(14, 7))
sns.barplot(x=top_n.values, y=top_n.index)
plt.title("Top Neighborhoods by Barrier Count")
plt.xlabel("Count")
plt.ylabel("")
plt.show()


In [ ]:
#Mobility friction score
neigh = (df
    .groupby("neighborhood")
    .agg(
        barriers=("attribute_id", "count"),
        avg_severity=("severity", "mean"),
        pct_temporary=("is_temporary", "mean"),
        severe_share=("severity", lambda s: (s >= 4).mean())
    )
    .reset_index()
)

# Example friction score: count * avg severity * permanence weight
neigh["permanence_weight"] = 1 + (1 - neigh["pct_temporary"])  # more permanent -> higher
neigh["friction_score"] = neigh["barriers"] * neigh["avg_severity"] * neigh["permanence_weight"]

neigh.sort_values("friction_score", ascending=False).head(10)


In [ ]:
#Plot top neighborhoods by friction score
top_f = neigh.sort_values("friction_score", ascending=False).head(15)

plt.figure(figsize=(14, 7))
sns.barplot(data=top_f, x="friction_score", y="neighborhood")
plt.title("Top Neighborhoods by Mobility Friction Score")
plt.xlabel("Friction Score (severity × count × permanence weight)")
plt.ylabel("")
plt.show()


In [ ]:
#Static Density Map
plt.figure(figsize=(10, 10))
plt.hexbin(df["lon"], df["lat"], gridsize=80, mincnt=1)
plt.title("Barrier Density (Hexbin)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
#Severity weighted density
plt.figure(figsize=(10, 10))
plt.hexbin(df["lon"], df["lat"], C=df["severity"], reduce_C_function=np.mean, gridsize=80, mincnt=5)
plt.title("Average Severity by Location (Hexbin)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
!pip -q install folium


In [ ]:
#Interactive heat map
import folium
from folium.plugins import HeatMap

center = [df["lat"].mean(), df["lon"].mean()]
m = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

# Heatmap: weight by severity (higher severity = "hotter")
heat_data = df[["lat", "lon", "severity"]].dropna().values.tolist()
HeatMap(heat_data, radius=10, blur=15, max_zoom=13).add_to(m)
m.save("interactive_heat_map.html")
m

In [ ]:
#Interactive clustered markers (click + inspect)
from folium.plugins import MarkerCluster

m2 = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")
cluster = MarkerCluster().add_to(m2)

sample = df.dropna(subset=["lat", "lon"]).sample(min(3000, len(df)), random_state=7)

for _, r in sample.iterrows():
    popup = (f"Type: {r['label_type']}<br>"
             f"Neighborhood: {r['neighborhood']}<br>"
             f"Severity: {r['severity']}<br>"
             f"Temporary: {r['is_temporary']}")
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=3,
        popup=popup,
        fill=True
    ).add_to(cluster)
m2.save("clustered_markers_map.html")
m2


In [ ]:
#Neighborhood barrier type composition 
top_neighs = df["neighborhood"].value_counts().head(8).index
top_types = df["label_type"].value_counts().head(6).index

pivot = (df[df["neighborhood"].isin(top_neighs) & df["label_type"].isin(top_types)]
         .pivot_table(index="neighborhood", columns="label_type", values="attribute_id", aggfunc="count", fill_value=0))

pivot = pivot.loc[top_neighs]  # keep order

pivot.plot(kind="bar", stacked=True, figsize=(14, 7))
plt.title("Barrier Type Mix (Top Neighborhoods)")
plt.xlabel("")
plt.ylabel("Count")
plt.xticks(rotation=30, ha="right")
plt.legend(title="Barrier Type", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
#Worst clusters (top severe points)
worst = df.dropna(subset=["severity"]).sort_values("severity", ascending=False).head(20)
worst[["label_type", "neighborhood", "severity", "lat", "lon", "is_temporary"]]
